In [ ]:
from matplotlib.pyplot import *
from numpy import *
import numpy.random as npr
rcParams['text.usetex'] = True

In [ ]:
from optics_design_workbench import distributions
from optics_design_workbench.distributions import *

# Test smoothnes warning

In [ ]:
x = distributions.ScalarRandomVariable('exp(-x**2)*cos(x)**2', variableDomain=(0,3))
x.draw(), x.mode()

In [ ]:
x = distributions.ScalarRandomVariable('1/x**2', variableDomain=(-1,1))
x.draw(), x.mode()

# Test random number generator API

In [ ]:
for x in [
  distributions.ScalarRandomVariable('x', variable='x', variableDomain=[0,1.5]),
  distributions.ScalarRandomVariable('1', variable='x', variableDomain=[0,1.5]),
  distributions.ScalarRandomVariable('exp(-x**2)', variable='x', variableDomain=[0,3]),
  distributions.ScalarRandomVariable('exp(-x**2)*sin(x)', variable='x', variableDomain=[0,3]),
]:
  X = x.draw(N=1e3)
  figure(figsize=(4,1.5))
  hist(X, bins=50)
  title(f'true random (N={len(X)})')
  
  X = x.drawPseudo(N=1e3, bins=50)
  figure(figsize=(4,1.5))
  hist(X, bins=50)
  title(f'pseudo random (N={len(X)})')

  X = x.findGrid(N=30)
  figure(figsize=(4,1.5))
  plot(X, 0*X, 'x')
  title('"fan" mode')


In [ ]:
x = distributions.VectorRandomVariable('x - y', variableOrder=['x', 'y'], variableDomains=dict(x=(1,2), y=(0,1)))
figure(figsize=(4,1.5))
X, Y = x.draw(N=1e3)
hist2d(X, Y, bins=5)
print(X.shape)

figure(figsize=(4,1.5))
X, Y = x.drawPseudo(N=1e3)
hist2d(X, Y, bins=5)
print(X.shape)

# Test 1D point generators for given densities (ray fans)

In [ ]:
Xrand = npr.normal(5, 1, size=150)
hX, hDensity = calcHistDensity(Xrand)
dX, dDensity = calcDiffDensity(Xrand)
plot(hX, hDensity, '.')
plot(dX, dDensity, '.')
xlabel('x')
ylabel('density')

### Gaussian

In [ ]:
X = linspace(-1, 2, 500)
Y = exp(-5*X**2)
Xgen = generatePointsWithGivenDensity1D(density=(X,Y), N=20)
dX, dDens = calcDiffDensity(Xgen)

In [ ]:
plot(X, Y/max(Y), label=r'beliebige Verteilungsfunktion\\als X,Y arrays angegeben')
xlabel('x')
ylabel('normierte Dichte')
plot(Xgen, [0]*len(Xgen), 'o', label=r'platzierte Punkte')
plot(dX, dDens/max(dDens), 'x', label=r'aus den Punktabst\"anden\\bestimmte Dichte')
legend()
#io.savefig('~/Desktop/gaussian-platzierung.jpg')

In [ ]:
rmsErr = sqrt(mean([abs((Y/max(Y))[argmin(abs(X-x))]-y)**2 for x,y in zip(dX, dDens/max(dDens))]))
print(rmsErr)
assert rmsErr < 1e-2

### Tophat

In [ ]:
X = linspace(-1, 2, 500)
Y = arctan( 1e5 * ( exp(-5*X**2) - .5 ) )/pi + .5
Xgen = generatePointsWithGivenDensity1D(density=(X,Y), N=10)
dX, dDens = calcDiffDensity(Xgen)

In [ ]:
plot(X, Y/max(Y), label=r'given density')
xlabel('x')
ylabel('normalized density')
plot(Xgen, [0]*len(Xgen), 'o', label=r'generated points')
plot(dX, dDens/max(dDens), 'x', label=r'\parbox{4cm}{density reconstructed from point-distance}')
legend()

In [ ]:
rmsErr = sqrt(mean([abs((Y/max(Y))[argmin(abs(X-x))]-y)**2 for x,y in zip(dX, dDens/max(dDens))]))
print(rmsErr)
assert rmsErr < 1e-5

### tests distorted gaussian

In [ ]:
X = linspace(-1, 3, 500)
Y = arctan( 20*exp(-5*X**2) ) * (1+X)
Xgen = generatePointsWithGivenDensity1D(density=(X,Y), N=25)
dX, dDens = calcDiffDensity(Xgen)

In [ ]:
plot(X, Y/max(Y), label=r'given density')
xlabel('x')
ylabel('normalized density')
plot(Xgen, [0]*len(Xgen), 'o', label=r'generated points')
plot(dX, dDens/max(dDens), 'x', label=r'\parbox{4cm}{density reconstructed from point-distance}')
legend()

In [ ]:
rmsErr = sqrt(mean([abs((Y/max(Y))[argmin(abs(X-x))]-y)**2 for x,y in zip(dX, dDens/max(dDens))]))
print(rmsErr)
assert rmsErr < 1e-2

### Test distribution with multiple maxima

In [ ]:
X = linspace(-1, 5, 500)
Y = exp(-5*X**2) + 0.7*exp(-5*(X-2)**2) 
Xgen = generatePointsWithGivenDensity1D(density=(X,Y), N=30)
dX, dDens = calcDiffDensity(Xgen)

In [ ]:
plot(X, Y/max(Y), label=r'given density')
xlabel('x')
ylabel('normalized density')
plot(Xgen, [0]*len(Xgen), 'o', label=r'generated points')
plot(dX, dDens/max(dDens), 'x', label=r'\parbox{4cm}{density reconstructed from point-distance}')
legend()

In [ ]:
rmsErr = sqrt(mean([abs((Y/max(Y))[argmin(abs(X-x))]-y)**2 for x,y in zip(dX, dDens/max(dDens))]))
print(rmsErr)
assert rmsErr < 3e-2

### same with more points

In [ ]:
X = linspace(-1, 5, 500)
Y = exp(-5*X**2) + 0.7*exp(-5*(X-2)**2)
Xgen = generatePointsWithGivenDensity1D(density=(X,Y), N=90)
dX, dDens = calcDiffDensity(Xgen)

In [ ]:
plot(X, Y/max(Y), label=r'given density')
xlabel('x')
ylabel('normalized density')
plot(Xgen, [0]*len(Xgen), 'o', label=r'generated points')
plot(dX, dDens/max(dDens), 'x', label=r'\parbox{4cm}{density reconstructed from point-distance}')
legend()

In [ ]:
rmsErr = sqrt(mean([abs((Y/max(Y))[argmin(abs(X-x))]-y)**2 for x,y in zip(dX, dDens/max(dDens))]))
print(rmsErr)
assert rmsErr < 1e-2